In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# suppress warning message
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import os
import pyarrow.compute as pc
from ds_capability import FeatureSelect, FeatureEngineer, AutoML, Controller

## Synthesize

In [4]:
syn_fe = FeatureEngineer.from_memory()
syn_ds = FeatureSelect.from_memory()

syn_fe.add_connector_uri('sample', 'https://raw.githubusercontent.com/project-hadron/hadron-asset-bank/master/datasets/toy_sample/titanic.csv')

syn_fe.set_persist_uri('./hadron/data/synthetic_sample.parquet')

In [5]:
tbl = syn_fe.tools.get_analysis_group(size=2000, other='sample', group_by='survived')
tbl.shape

(2000, 14)

In [6]:
tbl = syn_ds.tools.auto_drop_columns(tbl, headers=['survived'])
tbl.shape

(2000, 13)

In [7]:
syn_fe.save_persist_canonical(tbl)

In [8]:
result = syn_fe.load_persist_canonical()
result.shape

(2000, 13)

## Remote Run

In [9]:
os.environ['HADRON_PREDICT_SOURCE_DATA'] = './hadron/data/synthetic_sample.parquet'
os.environ['HADRON_PREDICT_PERSIST_DATA'] = './hadron/data/hadron_docs_titanic_predict.parquet'
os.environ['HADRON_PM_REPO'] = 'https://raw.githubusercontent.com/project-hadron/hadron-asset-bank/master/contracts/pyarrow/docs/use_case_one/'

In [10]:
ctrl = Controller.from_env()

In [11]:
ctrl.run_controller()

## View Remote

In [12]:
AutoML.from_env('survived').load_persist_canonical()

pyarrow.Table
predict: int64
----
predict: [[1,0,0,0,0,...,0,0,1,1,1]]